In [16]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import regex as re
import csv

In [17]:
def clean_dataframe(df, team_names):
    if df.empty:
        return None
    
    df_copy = df.copy()

    # Rename the columns
    df_copy.columns = ['name', 'blank', 'rating', 'acs', 'kills', 'deaths', 'assists', 'k/d', 'KAST', 'adr', 'hs', 'fk', 'fd', 'fk/fd']

    # Clean the 'name' column
    df_copy['name'] = df_copy['name'].str.strip().str.replace('\t', '').str.replace('\n', '')
    
#     # Clean the team names
#     for team in team_names:
#         df_copy['name'] = df_copy['name'].str.replace(team, '').str.strip()

    # Drop all unneeded columns
    df_copy = df_copy.drop(columns=['blank', 'rating', 'acs', 'k/d', 'KAST', 'hs', 'fk/fd'])
    
    # Apply a lambda function to extract the first number from each cell
    df_copy['kills'] = df_copy['kills'].apply(lambda x: x.split('\n')[0] if x else None)

    # Use a try-except block to handle potential errors in 'deaths' column processing
    try:
        df_copy['deaths'] = df_copy['deaths'].apply(lambda x: int(re.findall(r'\d+', x)[0]) if x else None)
    except IndexError:
        df_copy['deaths'] = None  # Handle the error by assigning a default value

    df_copy['assists'] = df_copy['assists'].apply(lambda x: x.split('\n')[0] if x else None)
    df_copy['adr'] = df_copy['adr'].apply(lambda x: x.split('\n')[0] if x else None)
    df_copy['fk'] = df_copy['fk'].apply(lambda x: x.split('\n')[0] if x else None)
    df_copy['fd'] = df_copy['fd'].apply(lambda x: x.split('\n')[0] if x else None)

    return df_copy

def scrape_data(url_list):
    all_dfs = {}  # Dictionary to store processed DataFrames for each URL

    team_names = ['MIBR', 'LEV', 'SEN', 'NRG', 'FUR', '100T', 'LOUD', 'EG', 'G2', 'C9', 'KRÜ']

    for url in url_list:
        response = requests.get(url)
        if response.status_code == 200:
            soup = bs(response.content, 'html.parser')

            # Initialize lists to store DataFrames for each pass
            first_pass_dfs = []
            second_pass_dfs = []

            # Find all game divs
            game_divs = soup.find_all('div', class_='vm-stats-game')

            # First pass: Find initial tables
            for game_div in game_divs:
                table = game_div.find('table', class_='wf-table-inset mod-overview')

                if table:
                    # Extract table data into a DataFrame
                    table_data = []
                    rows = table.find_all('tr')
                    for row in rows:
                        row_data = [cell.text.strip() for cell in row.find_all(['td', 'th'])]
                        table_data.append(row_data)

                    # Convert table_data into a DataFrame and append to first_pass_dfs list
                    df = pd.DataFrame(table_data[1:], columns=table_data[0])  # Assuming first row is header
                    first_pass_dfs.append(df)

            # Second pass: Find the next tables
            for game_div in game_divs:
                table = game_div.find('table', class_='wf-table-inset mod-overview')
                if table:
                    next_table = table.find_next('table', class_='wf-table-inset mod-overview')
                    if next_table:
                        # Extract table data into a DataFrame
                        table_data = []
                        rows = next_table.find_all('tr')
                        for row in rows:
                            row_data = [cell.text.strip() for cell in row.find_all(['td', 'th'])]
                            table_data.append(row_data)

                        # Convert table_data into a DataFrame and append to second_pass_dfs list
                        df = pd.DataFrame(table_data[1:], columns=table_data[0])  # Assuming first row is header
                        second_pass_dfs.append(df)

            # Process and clean DataFrames from both passes
            first_pass_cleaned = [clean_dataframe(df, team_names) for df in first_pass_dfs if not df.empty]
            second_pass_cleaned = [clean_dataframe(df, team_names) for df in second_pass_dfs if not df.empty]

            # Combine corresponding DataFrames from both passes
            combined_dfs = []
            min_length = min(len(first_pass_cleaned), len(second_pass_cleaned))
            for i in range(min_length):
                if first_pass_cleaned[i] is not None and second_pass_cleaned[i] is not None:
                    combined_df = pd.concat([first_pass_cleaned[i], second_pass_cleaned[i]], axis=0)
                    combined_dfs.append(combined_df)
                    combined_df.reset_index(inplace=True, drop=True)

            all_dfs[url] = combined_dfs

        else:
            print('Failed to retrieve the webpage. Status code:', response.status_code)

    return all_dfs

In [18]:
# Example usage
url_list = [
 'https://www.vlr.gg/353177/mibr-vs-leviat-n-champions-tour-2024-americas-stage-2-w1/?game=all&tab=overview',
 'https://www.vlr.gg/353178/sentinels-vs-nrg-esports-champions-tour-2024-americas-stage-2-w1/?game=all&tab=overview',
 'https://www.vlr.gg/353179/furia-vs-100-thieves-champions-tour-2024-americas-stage-2-w1/?game=all&tab=overview',
 'https://www.vlr.gg/353180/loud-vs-evil-geniuses-champions-tour-2024-americas-stage-2-w1/?game=all&tab=overview',
 'https://www.vlr.gg/353181/g2-esports-vs-cloud9-champions-tour-2024-americas-stage-2-w1/?game=all&tab=overview',
 'https://www.vlr.gg/353182/evil-geniuses-vs-furia-champions-tour-2024-americas-stage-2-w2/?game=all&tab=overview',
 'https://www.vlr.gg/353183/sentinels-vs-kr-esports-champions-tour-2024-americas-stage-2-w2/?game=all&tab=overview',
 'https://www.vlr.gg/353184/loud-vs-mibr-champions-tour-2024-americas-stage-2-w2/?game=all&tab=overview',
 'https://www.vlr.gg/353185/leviat-n-vs-100-thieves-champions-tour-2024-americas-stage-2-w2/?game=all&tab=overview',
 'https://www.vlr.gg/353186/nrg-esports-vs-g2-esports-champions-tour-2024-americas-stage-2-w2/?game=all&tab=overview',
 'https://www.vlr.gg/353187/sentinels-vs-cloud9-champions-tour-2024-americas-stage-2-w2/?game=all&tab=overview',
 'https://www.vlr.gg/353188/evil-geniuses-vs-leviat-n-champions-tour-2024-americas-stage-2-w2/?game=all&tab=overview',
 'https://www.vlr.gg/353189/loud-vs-furia-champions-tour-2024-americas-stage-2-w2/?game=all&tab=overview',
 'https://www.vlr.gg/353190/mibr-vs-100-thieves-champions-tour-2024-americas-stage-2-w2/?game=all&tab=overview',
 'https://www.vlr.gg/353191/nrg-esports-vs-kr-esports-champions-tour-2024-americas-stage-2-w2/?game=all&tab=overview',
 'https://www.vlr.gg/353192/g2-esports-vs-kr-esports-champions-tour-2024-americas-stage-2-w3/?game=all&tab=overview',
 'https://www.vlr.gg/353193/loud-vs-leviat-n-champions-tour-2024-americas-stage-2-w3/?game=all&tab=overview',
 'https://www.vlr.gg/353194/mibr-vs-furia-champions-tour-2024-americas-stage-2-w3/?game=all&tab=overview',
 'https://www.vlr.gg/353195/nrg-esports-vs-cloud9-champions-tour-2024-americas-stage-2-w3/?game=all&tab=overview',
 'https://www.vlr.gg/353196/evil-geniuses-vs-100-thieves-champions-tour-2024-americas-stage-2-w3/?game=all&tab=overview',
 'https://www.vlr.gg/353197/cloud9-vs-kr-esports-champions-tour-2024-americas-stage-2-w4/?game=all&tab=overview',
 'https://www.vlr.gg/353198/evil-geniuses-vs-mibr-champions-tour-2024-americas-stage-2-w4/?game=all&tab=overview',
 'https://www.vlr.gg/353199/leviat-n-vs-furia-champions-tour-2024-americas-stage-2-w4/?game=all&tab=overview',
 'https://www.vlr.gg/353200/loud-vs-100-thieves-champions-tour-2024-americas-stage-2-w4/?game=all&tab=overview',
 'https://www.vlr.gg/353201/sentinels-vs-g2-esports-champions-tour-2024-americas-stage-2-w4/?game=all&tab=overview'
]

data_frames = scrape_data(url_list)

In [19]:
# remove the second df from each series

# Create a list of new keys
new_keys = [f'Series {i+1}' for i in range(len(data_frames))]

# Create a new dictionary with updated keys
re_dfs = dict(zip(new_keys, data_frames.values()))

In [20]:
re_dfs['Series 2']

[           name kills  deaths assists  adr fk fd
 0    zekken SEN    22      14       7  222  7  4
 1      Sacy SEN    19      10       9  161  0  2
 2   Zellsis SEN    16      10       6  131  1  0
 3      TenZ SEN    13      13      14  127  1  1
 4    johnqt SEN    10      11       5   96  2  2
 5       s0m NRG    14      17       6  142  4  4
 6     Ethan NRG    14      15      10  121  3  0
 7    Victor NRG    11      17       5  116  2  6
 8   FiNESSE NRG     9      16       4   79  1  0
 9  crashies NRG    10      16       4   92  0  2,
            name kills  deaths assists  adr  fk  fd
 0    zekken SEN    46      33      21  213  12   8
 1   Zellsis SEN    37      24      22  135   2   1
 2      TenZ SEN    36      29      24  134   5   3
 3      Sacy SEN    38      26      17  145   3   5
 4    johnqt SEN    25      28       9  102   3   4
 5       s0m NRG    30      35      21  131   4   6
 6     Ethan NRG    32      34      23  118   5   3
 7    Victor NRG    34      41   

In [21]:
# Iterate through the dictionary and remove the second item from each list value
for key in re_dfs:
    if len(re_dfs[key]) > 1:
        del re_dfs[key][1]  # Delete the second item (index 1)

In [22]:
re_dfs['Series 2'] 

[           name kills  deaths assists  adr fk fd
 0    zekken SEN    22      14       7  222  7  4
 1      Sacy SEN    19      10       9  161  0  2
 2   Zellsis SEN    16      10       6  131  1  0
 3      TenZ SEN    13      13      14  127  1  1
 4    johnqt SEN    10      11       5   96  2  2
 5       s0m NRG    14      17       6  142  4  4
 6     Ethan NRG    14      15      10  121  3  0
 7    Victor NRG    11      17       5  116  2  6
 8   FiNESSE NRG     9      16       4   79  1  0
 9  crashies NRG    10      16       4   92  0  2,
            name kills  deaths assists  adr fk fd
 0      TenZ SEN    23      16      10  140  4  2
 1    zekken SEN    24      19      14  206  5  4
 2   Zellsis SEN    21      14      16  138  1  1
 3      Sacy SEN    19      16       8  132  3  3
 4    johnqt SEN    15      17       4  108  1  2
 5     Ethan NRG    18      19      13  117  2  3
 6       s0m NRG    16      18      15  123  0  2
 7    Victor NRG    23      24       5  177  7  7